<a href="https://colab.research.google.com/github/RodrigoLima82/competicao_dsa_jun_2019/blob/master/Solution_Competition_DSA_Jun_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Competição DSA de Machine Learning - Edição Junho/2019
Imagine estar com fome em uma parte desconhecida da cidade e receber recomendações de restaurantes, com base em suas preferências pessoais, no momento certo. A recomendação vem com um desconto em anexo da sua operadora de cartão de crédito para um local ao virar a esquina!

Uma Startup pensou nisso e construiu parcerias com comerciantes para oferecer promoções ou descontos aos portadores de cartões de crédito. Mas essas promoções funcionam tanto para o consumidor quanto para o comerciante? Os clientes aproveitam a experiência? Os comerciantes veem resultado? A personalização é fundamental.

Os profissionais da Startup construíram modelos de aprendizado de máquina para entender os aspectos e preferências mais importantes no ciclo de vida de seus clientes, desde alimentos a compras. Mas até agora nenhum deles é especificamente adaptado para um indivíduo ou perfil. É aqui que você entra. Precisando de um modelo preditivo mais robusto, a Startup selecionou você como Cientista de Dados.


## The Goal
 
- Os arquivos dataset_treino.csv e dataset_teste.csv contêm card_ids e informações sobre o próprio cartão - o primeiro mês em que o cartão estava ativo, etc.
- Objetivo é prever um índice de lealdade para cada card_id
- O modelo é avalidado pelo Root-Mean-Squared-Error (RMSE)

## Key features of the model training process in this kernel:
- **Cross Validation:** Using 12-fold cross-validation
- **Models:** On each run of cross-validation I fit 7 models (ridge, svr, gradient boosting, random forest, xgboost, lightgbm regressors)
- **Stacking:** In addition, I trained a meta StackingCVRegressor optimized using xgboost
- **Blending:** All models trained will overfit the training data to varying degrees. Therefore, to make final predictions, I blended their predictions together to get more robust predictions.

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Importando bibliotecas que serao utilizadas neste projeto
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesClassifier
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

# Misc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import utils

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

import os
print(os.listdir("/content/gdrive/My Drive/kaggle/data"))

In [0]:
# Read in the dataset as a dataframe
train = pd.read_csv('/content/gdrive/My Drive/kaggle/data/dataset_treino.csv', low_memory=False)
com = pd.read_csv('/content/gdrive/My Drive/kaggle/data/comerciantes.csv', low_memory=False)

In [0]:
import zipfile

zf = zipfile.ZipFile('/content/gdrive/My Drive/kaggle/data/transacoes_historicas.zip')
hist = pd.read_csv(zf.open('transacoes_historicas.csv'), nrows=10000000)

In [0]:
#import zipfile

#zf = zipfile.ZipFile('/content/gdrive/My Drive/kaggle/data/novas_transacoes_comerciantes.zip')
#novas = pd.read_csv(zf.open('novas_transacoes_comerciantes.csv'),)

In [0]:
# Uniao dos dataset de treino e transacoes novas e historicas
df = pd.merge(train, hist, on='card_id')

In [0]:
# Uniao dos dataset de treino e comerciantes
df = pd.merge(df, com, on='merchant_id')

In [0]:
# Uniao dos dataset de treino e comerciantes
#df = pd.merge(df, novas, on='merchant_id')

In [0]:
# Removendo df temporarios
import gc 

del train, hist, com

gc.collect()

In [0]:
# Transformando algumas variaveis string para inteiro
df['new_authorized_flag'] = df['authorized_flag'].replace(to_replace=['N', 'Y'], value=[0, 1])
df['new_category_3'] = df['category_3'].replace(to_replace=['A', 'B', 'C'], value=[0, 1, 2])

# Removendo colunas nao utilizadas
df.drop(columns =["merchant_id", "purchase_date", "first_active_month", "authorized_flag", "category_1_x", "category_1_y", "category_3", "city_id_y", "state_id_y", "category_2_y", "merchant_category_id_y", "subsector_id_y", "category_4", "most_recent_sales_range", "most_recent_purchases_range"], inplace = True) 


In [0]:
# log(1+x) transform
df["target"] = np.log1p(df["target"])

In [0]:
# drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in the dataset
print(df.shape)

In [0]:
# determine the threshold for missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(df)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:26]

In [0]:
df['category_2_x'].fillna(0, inplace=True)
df['merchant_group_id'].fillna(0, inplace=True)
df['category_2_x'].fillna(df['category_2_x'].median(), inplace=True)
df['merchant_group_id'].fillna(df['merchant_group_id'].median(), inplace=True)
df['new_category_3'].fillna(df['new_category_3'].median(), inplace=True)
df['active_months_lag3'].fillna(df['active_months_lag3'].median(), inplace=True)
df['avg_sales_lag6'].fillna(df['avg_sales_lag6'].median(), inplace=True)
df['avg_purchases_lag6'].fillna(df['avg_purchases_lag6'].median(), inplace=True)
df['active_months_lag6'].fillna(df['active_months_lag6'].median(), inplace=True)
df['avg_purchases_lag12'].fillna(df['avg_purchases_lag12'].median(), inplace=True)
df['active_months_lag12'].fillna(df['active_months_lag12'].median(), inplace=True)

Criação e Validação dos Modelos de Machine Learning

In [11]:
# Criacao dos modelos de Regressao

# Separando o array em componentes de input e output
X = df.drop(['card_id'], axis=1)
Y = df['target'].reset_index(drop=True)

# Aplicando a mesma escala nos dados
X = MinMaxScaler().fit_transform(X)

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
X = StandardScaler().fit_transform(X)

# Definindo os valores para o número de folds
num_folds = 3
seed = 10

# Preparando a lista de modelos
modelos = []
modelos.append(('lightgbm', LGBMRegressor()))
modelos.append(('xgboost', XGBRegressor()))
modelos.append(('gbr', GradientBoostingRegressor()))
modelos.append(('rf', RandomForestRegressor()))
modelos.append(('SVM', SVR()))

# Avaliando cada modelo em um loop
resultados = []
nomes = []

for nome, modelo in modelos:
    kfold = KFold(n_splits = num_folds, random_state = seed)
    cv_results = np.sqrt(-cross_val_score(modelo, X, Y, cv = kfold, scoring="neg_mean_squared_error"))
    resultados.append(cv_results)
    nomes.append(nome)
    msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std())
    print(msg)

# Boxplot para comparar os algoritmos
fig = plt.figure()
fig.suptitle('Comparacao de Algoritmos de Regressao')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nomes)
plt.show()


# versao 1
#lightgbm: 0.858902 (0.192160)
#xgboost: 0.820020 (0.200089)
#gbr: 0.819662 (0.198220)
#rf: 0.983438 (0.143144)

ValueError: ignored

In [0]:
# Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X_teste, Y_teste, scoring="neg_mean_squared_error", cv=10))
    return (rmse)

In [32]:
# Separando o array em componentes de input e output
X = df.drop(['card_id','target'], axis=1)
Y = df['target'].reset_index(drop=True)

# Aplicando a mesma escala nos dados
X = MinMaxScaler().fit_transform(X)

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
X = StandardScaler().fit_transform(X)

# Definindo os valores para o número de folds
num_folds = 0.33
seed = 10

# Criando o dataset de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = num_folds, random_state = seed)

# Regressor GBRT
modeloGBR = GradientBoostingRegressor(n_estimators = 1000, max_depth = 1, learning_rate = 1.0)

# Treinando o modelo
modeloGBR.fit(X_treino, Y_treino)


KeyboardInterrupt: ignored

In [28]:
score = cv_rmse(modeloGBR)
print("RSME: {:.4f} ({:.4f})".format(score.mean(), score.std()))

RSME: 1.0046 (0.1443)


In [31]:
# Separando o array em componentes de input e output
X = df.drop(['card_id','target'], axis=1)
Y = df['target'].reset_index(drop=True)

# Aplicando a mesma escala nos dados
X = MinMaxScaler().fit_transform(X)

# Padronizando os dados (0 para a média, 1 para o desvio padrão)
X = StandardScaler().fit_transform(X)

# Definindo os valores para o número de folds
num_folds = 0.33
seed = 10

# Criando o dataset de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = num_folds, random_state = seed)

# Grid de parâmetros
param_grid = {'learning_rate': [0.1, 0.01, 0.001],
              'max_depth': [4, 6],
              'min_samples_leaf': [3, 5]
              }

# Regressor
modeloGBR_v2 = GradientBoostingRegressor(n_estimators = 3000)

# Modelo criado com GridSearchCV
gs_cv = GridSearchCV(modeloGBR_v2, param_grid, scoring = 'neg_mean_squared_error', n_jobs = 4).fit(X_treino, Y_treino)

# Imprime os melhors parâmetros
print('Melhores Hyperparametros: %r' % gs_cv.best_params_)

KeyboardInterrupt: ignored

In [0]:
# Recria o modelo com os melhores parâmetros
modeloGBR_v2.set_params(**gs_cv.best_params_)
modeloGBR_v2.fit(X_treino, Y_treino)

In [0]:
score = cv_rmse(modeloGBR_v2)
print("RSME: {:.4f} ({:.4f})".format(score.mean(), score.std()))